In [1]:
import sys
import os
import ast
from snowflake.nodes import *
from snowflake.stencil_compiler import PythonCompiler, CCompiler
from ctree.frontend import dump
import numpy as np
import inspect
!ctree -cc

ctree looking for relative cache directories named compiled, checking directories under this one



In [2]:
compiler = PythonCompiler()

In [3]:
arr = WeightArray([
    [1/9.0, 1/9.0, 1/9.0],
    [1/9.0, 1/9.0, 1/9.0],
    [1/9.0, 1/9.0, 1/9.0]
])
#arr2 = WeightArray([0])
s = StencilComponent("a", arr)
simple_stencil = Stencil(s, "out", ((1, -1), (1, -1)))

In [4]:
output = np.zeros((8, 8), dtype=np.double)

In [5]:
output

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [6]:
kernel = compiler.compile(simple_stencil)

In [7]:
test_data = np.arange(64, dtype=np.double).reshape(8, 8)

In [8]:
kernel(output, test_data)

In [9]:
output

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   9.,  10.,  11.,  12.,  13.,  14.,   0.],
       [  0.,  17.,  18.,  19.,  20.,  21.,  22.,   0.],
       [  0.,  25.,  26.,  27.,  28.,  29.,  30.,   0.],
       [  0.,  33.,  34.,  35.,  36.,  37.,  38.,   0.],
       [  0.,  41.,  42.,  43.,  44.,  45.,  46.,   0.],
       [  0.,  49.,  50.,  51.,  52.,  53.,  54.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]])

In [10]:
test_data

array([[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.],
       [  8.,   9.,  10.,  11.,  12.,  13.,  14.,  15.],
       [ 16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.],
       [ 24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.],
       [ 32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.],
       [ 40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.],
       [ 48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.],
       [ 56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.]])

In [11]:
ccompiler = CCompiler()

In [12]:
ckernel = ccompiler.compile(simple_stencil)

In [13]:
output[:] = 0
ckernel(output, test_data)

In [14]:
output

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   9.,  10.,  11.,  12.,  13.,  14.,   0.],
       [  0.,  17.,  18.,  19.,  20.,  21.,  22.,   0.],
       [  0.,  25.,  26.,  27.,  28.,  29.,  30.,   0.],
       [  0.,  33.,  34.,  35.,  36.,  37.,  38.,   0.],
       [  0.,  41.,  42.,  43.,  44.,  45.,  46.,   0.],
       [  0.,  49.,  50.,  51.,  52.,  53.,  54.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]])

In [15]:
output.shape

(8, 8)

In [16]:
restriction_stencil = ScalingStencil(
    StencilComponent('a', WeightArray([1])),
    'out',
    ((1, -1, 1),),
    (1,),
    (1,),
    (0.5,)
)

In [17]:
print(dump(restriction_stencil))

ScalingStencil(op_tree=StencilComponent(name='a', weights=WeightArray(weights=[
    StencilConstant(value=1),
  ])))


In [18]:
restriction_kernel = ccompiler.compile(restriction_stencil)

In [19]:
data1 = np.arange(10)

In [20]:
output = np.zeros(6)

In [21]:
restriction_kernel(output, data1)

In [22]:
output

array([ 0.,  2.,  4.,  6.,  8.,  0.])

In [23]:
n_average_rs = ScalingStencil(
    StencilComponent('a', WeightArray(
            [
                [1.0/16, 1.0/8, 1.0/16],
                [1.0/8, 1.0/4, 1.0/8],
                [1.0/16, 1.0/8, 1.0/16]
            ]
        )),
    'out',
    ((1, -1, 1), (1, -1, 1)),
    (1,1),
    (1,1),
    (0.5, 0.5)
)

In [24]:
n_average_kern = ccompiler.compile(n_average_rs)

In [25]:
data2d = np.arange(100).reshape(10, 10)

In [26]:
output = np.zeros((6, 6))

In [27]:
n_average_kern(output, data2d)

In [28]:
output

array([[  0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,  22.,  24.,  26.,  28.,   0.],
       [  0.,  42.,  44.,  46.,  48.,   0.],
       [  0.,  62.,  64.,  66.,  68.,   0.],
       [  0.,  82.,  84.,  86.,  88.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.]])

In [29]:
data2d

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
       [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
       [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
       [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
       [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
       [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]])